In [ ]:
%load_ext autoreload
%autoreload 2
%pylab inline

In [ ]:
%load_ext Cython
%load_ext line_profiler
%load_ext memory_profiler

## Import framework main libs

In [ ]:
from tmqrfeed.manager import DataManager
from tmqrstrategy.optimizers import OptimizerBase, OptimizerGenetic

## Import strategy from module

In [ ]:
from tmqralphas.strategy_dsp_bandpass import Strategy_DSP_BandPass

In [ ]:
from tmqrstrategy import StrategyAlpha
import pandas as pd
import numpy as np


def CrossUp(a, b):
    """
    A crosses up B
    """
    return (a.shift(1) < b.shift(1)) & (a > b)


def CrossDown(a, b):
    """
    A crosses down B
    """
    return (a.shift(1) > b.shift(1)) & (a < b)


class Strategy_VWAP_TBF(StrategyAlpha):
    def calc_entryexit_rules(self, rolling_window_period, rule_index):
        try:
            exo_df = self.dm.quotes()
        except KeyError:
            # In case of index based quotes
            raise Exception('Use only with ContFut!')
            #px_ser = self.dm.quotes()['equity_decision']

        high_rollingmax = exo_df.h.rolling(int(rolling_window_period)).max()
        low_rollingmin = exo_df.l.rolling(int(rolling_window_period)).min()

        topfinder_ser = pd.Series(index=exo_df.index)
        botfinder_ser = pd.Series(index=exo_df.index)

        highest_high = exo_df.h >= high_rollingmax
        lowest_low = exo_df.l <= low_rollingmin

        highest_high = highest_high[highest_high == True]
        lowest_low = lowest_low[lowest_low == True]

        for i in range(highest_high.size):
            if highest_high[i] == True:
                if i == 0:
                    midas_close = (exo_df.h.ix[:highest_high.index[i]] + exo_df.l.ix[:highest_high.index[i]]) / 2
                    midas_volume = exo_df.v.ix[:highest_high.index[i]]

                    price_volume_onstart = pd.Series((midas_close[0] * midas_volume[0]), index=midas_close.index)
                    volume_onstart = pd.Series(midas_volume[0], index=midas_close.index)

                    midas_vwap = (((midas_close * midas_volume).cumsum() - price_volume_onstart) / 
                                  (midas_volume.cumsum() - volume_onstart))

                    topfinder_ser[midas_vwap.index] = midas_vwap

                else:
                    midas_close = (exo_df.h.ix[highest_high.index[i-1]: highest_high.index[i]] + 
                                   exo_df.l.ix[highest_high.index[i-1]: highest_high.index[i]]) / 2

                    midas_volume = exo_df.v.ix[highest_high.index[i-1]: highest_high.index[i]]

                    price_volume_onstart = pd.Series((midas_close[0] * midas_volume[0]), index=midas_close.index)
                    volume_onstart = pd.Series(midas_volume[0], index=midas_close.index)

                    midas_vwap = (((midas_close * midas_volume).cumsum() - price_volume_onstart) / 
                                  (midas_volume.cumsum() - volume_onstart))


                    topfinder_ser[midas_vwap.index] = midas_vwap


        for i in range(lowest_low.size):
            if lowest_low[i] == True:
                if i == 0:
                    midas_close = (exo_df.h.ix[:lowest_low.index[i]] + exo_df.l.ix[:lowest_low.index[i]]) / 2
                    midas_volume = exo_df.v.ix[:lowest_low.index[i]]

                    price_volume_onstart = pd.Series((midas_close[0] * midas_volume[0]), index=midas_close.index)
                    volume_onstart = pd.Series(midas_volume[0], index=midas_close.index)

                    midas_vwap = (((midas_close * midas_volume).cumsum() - price_volume_onstart) / 
                                  (midas_volume.cumsum() - volume_onstart))

                    topfinder_ser[midas_vwap.index] = midas_vwap

                else:
                    midas_close = (exo_df.h.ix[lowest_low.index[i-1]: lowest_low.index[i]] + 
                                   exo_df.l.ix[lowest_low.index[i-1]: lowest_low.index[i]]) / 2

                    midas_volume = exo_df.v.ix[lowest_low.index[i-1]: lowest_low.index[i]]

                    price_volume_onstart = pd.Series((midas_close[0] * midas_volume[0]), index=midas_close.index)
                    volume_onstart = pd.Series(midas_volume[0], index=midas_close.index)

                    midas_vwap = (((midas_close * midas_volume).cumsum() - price_volume_onstart) / 
                                  (midas_volume.cumsum() - volume_onstart))


                    botfinder_ser[midas_vwap.index] = midas_vwap

        avg_hl_price = (exo_df.h + exo_df.l) / 2            
        
        if rule_index == 0:
            entry_rule = CrossUp(avg_hl_price, topfinder_ser)
            exit_rule = CrossDown(avg_hl_price, botfinder_ser)

            return entry_rule, exit_rule


    def calculate(self, *args):
        (direction, rolling_window_period, rule_index) = args

        entry_rule, exit_rule = self.calc_entryexit_rules(rolling_window_period, rule_index)
        return self.exposure(entry_rule, exit_rule, direction)

In [ ]:
ALPHA_CONTEXT = {
        'name': 'ES_NewFramework_VWAP_TBF_Test', # Global alpha name, which be used for load/save from DB
        'context': { # Strategy specific settings
            # These settings only applycable for alphas derived from StrategyAlpha strategy 
            # StrategyAlpha - is a classic EXO/SmartEXO based alpha
            'index_name': 'US.ES_ContFutEOD',      # Name of EXO index to trade
            'costs_per_option': 3.0,
            'costs_per_contract': 3.0,
        },
        'wfo_params': {
            'window_type': 'rolling',  # Rolling window for IIS values: rolling or expanding
            'period': 'M',  # Period of rolling window 'M' - monthly or 'W' - weekly
            'oos_periods': 2,  # Number of months is OOS period
            'iis_periods': 12,
            # Number of months in IIS rolling window (only applicable for 'window_type' == 'rolling')
        },
        'wfo_optimizer_class': OptimizerGenetic, 
        'wfo_optimizer_class_kwargs': {
            'nbest_count': 3,
            'nbest_fitness_method': 'max',
            'population_size': 10, 
            'number_generations': 3, 
            # 'rand_seed': 1, # Uncomment this parameter to make genetic results repeatable
        },
        'wfo_opt_params': [
            ('direction', [1]),
            ('Rolling Window Period',  list(np.arange(10,200,10))),
            ('Rule index', [0]),
        ],
        'wfo_members_count': 1,
        'wfo_costs_per_contract': 0.0,
        'wfo_scoring_type': 'modsharpe'
    }

In [ ]:
list(np.arange(10,200,50))

# Run the alpha

In [ ]:
# DataManager is a core class of the framework
dm = DataManager()

# Init alpha class and run
alpha = Strategy_VWAP_TBF(dm, **ALPHA_CONTEXT)    

In [ ]:
alpha.run()

# Equity

In [ ]:
alpha.stats['series']['equity'].plot();

In [ ]:
type(alpha.stats)

In [ ]:
alpha.stats['series']['costs'].cumsum().plot()